# 제품 현황 분석

<br>

#### 가상의 앱
- 웰컴, 홈 화면, 검색

<img src='img/01-06.jpg' width=600>

<br>

- 음식 카테고리 ~ 주문 완료

<img src='img/01-07.jpg' width=600>

<br>

<hr>

## 앱 로그 데이터
- 데이터를 보는 관점에서 핵심 포인트
- Event
    - 사용자가 하는 행동 (클릭, 페이지 뷰 등)
    - 시스템 이벤트, 오류
    - 이벤트 파라미터 (Key), 이벤트 파라미터의 값 (Value)이 존재

<br>

#### 로그 설계
- https://zzsza.github.io/data/2021/06/13/data-event-log-definition/

<br>

<hr>

<br>

## ARRAY, STRUCT

<br>

### 데이터 확인
- 하나의 행에 여러 데이터가 포함된 형태 $\rightarrow$ ARRAY(배열), STRUCT(구조체)

<img src='img/02-01.png' width=600>


<br>

<img src='img/02-02.jpg' width=600>


<br>

### ARRAY (배열)
- 프로그래밍 언어에서 지원하는 자료형 중 하나
- **같은 타입의 여러 값을 하나의컬럼에 저장할 수 있는 자료형**
- BigQuery에서는 []를 사용하거나 ARRAY로 지정해서 쓸 수 있음
    - 배열로 저장할 때 저장 용량이 효율적

<br>

#### ARRAY(배열) 생성

1. **대괄호 `[]` 사용**

<img src='img/02-03.png' width=600>

<br>

2. **`ARRAY<>` 사용 : 예) `ARRAY<INT>`**

```sql
SELECT ARRAY<INT64>[0, 1, 3] AS some_numbers
```

3. **배열 생성 함수**

```sql
SELECT 
    GENERATE_DATE_ARRAY('2024-01-01', ..., INTERVAL 1 WEEK) AS output1,
    GENERATE_ARRAY(1, 5, 2) AS output 2
```

<br>

4. **`ARRAY_AGG`**
- 여러 결과를 마지막에 배열로 저장하고 싶은 경우

<img src='img/02-04.png' width=600>

<br>

#### 배열 데이터에 접근
- `OFFSET` : 0부터 시작
- `ORDINAL` : 1부터 시작

```sql
SELECT 배열_컬럼[OFFSET/ORDINAL(숫자)]
```

- **단, 배열의 길이보다 큰 값을 지정하면 오류 발생**

    **$\rightarrow$ 방지하기 위해 `SAFE_OFFSET()`을 사용**

<img src='img/02-05.jpg' width=600>

<br>

### STRUCT (구조체)
- **서로 다른 타입의 여러 값을 하나의 컬럼에 저장할 수 있는 자료형**

    $\rightarrow$ JSON, Dict 형태와 유사

<br>

#### STRUCT 생성
1. **소괄호`()` 사용**
    - 소괄호 사용시 이름이 지정되지 않음

```sql
SELECT (1, 2, 3) AS struct_test
```

<img src='img/02-06.png' width=600>

<br>

2. **`STRUCT<>()` 사용**

```sql
SELECT
    STRUCT<hi INT64, hello INT64, awesome STRING>(1, 2, 'HI') AS struct_test
```

<img src='img/02-07.png' width=600>

<br>

#### STRUCT 값에 접근
- **`STRUCT의 이름.key` 형태로 접근**

```sql
SELECT
    struct_test.hi,
    struct_test.hello
FROM (
    SELECT
        STRUCT<hi INT64, hello INT64, awesome STRING>(1, 2, 'HI') AS struct_test
)
```

<br>

### ARRAY, STRUCT의 관계
- **ARRAY 안에 STRUCT 저장 가능**
- **STRUCT 안에 ARRAY 저장 가능**
- **STRUCT 안에 STRUCT 저장 가능**
- **ARRAY 안에 ARRAY 저장 가능**

    $\rightarrow$ **중첩된 구조가 가능**

<br>

#### 예시 데이터 : 배열과 함께 존재

```sql
WITH example_data AS (
    SELECT
        'kyle' AS name,
        ['python', 'sql', 'r', 'julia', 'go'] AS preferred_language,
        'Incheon' AS hometown
    UNION ALL
    SELECT
        'max' AS name,
        ['python', 'sql', 'scala', 'java', 'kotlin'] AS preferred_language,
        'Seoul' AS hometown
    UNION ALL
    ...
)

SELECT * FROM example_data;
```

<img src='img/02-08.png' width=600>

<br>

### `UNNEST`
- 배열을 직접적으로 접근해서 사용하는 것 보다, 독립적인 행으로 풀어서 (평면화, Flatten) 사용

    $\rightarrow$ **ARRAY의 요소를 독립적인 행으로 펼칠 때 `UNNEST` 사용**

<img src='img/02-09.png' width=600>

<br>

#### `UNNEST` 사용 (Flatten)

- `UNNEST` 결과를 `CROSS JOIN`
    - `CROSS JOIN`은 생략하고 쉼표를 사용해도 괜찮음

```sql
SELECT
    a.column,
    alias_name,
FROM TABLE_A AS a
CROSS JOIN UNNEST(ARRAY_Column) AS alias_name
-- FROM TABLE_A AS a, UNNEST(ARRAY_Column) AS alias_name
```


<br>

- 평면화 예시

```sql
WITH example_data AS (
    SELECT
        'kyle' AS name,
        ['python', 'sql', 'r', 'julia', 'go'] AS preferred_language,
        'Incheon' AS hometown
    UNION ALL
    SELECT
        'max' AS name,
        ['python', 'sql', 'scala', 'java', 'kotlin'] AS preferred_language,
        'Seoul' AS hometown
    UNION ALL
    SELECT
      'yun' AS name,
      ['python', 'sql'] AS preferred_language,
      'Incheon' AS hometown
)

SELECT 
  name,
  pref_lang,
  hometown
FROM example_data
CROSS JOIN UNNEST(preferred_language) AS pref_lang;
```

<img src='img/02-10.png' width=600>

<br>


<br>

### `UNNEST` 예시

<br>

#### 테이블에서 `title`별로, `actor`, `character` `genre`를 출력
- 한 행에 `actor`, `character`, `genre`가 모두 표시


```sql
SELECT 
  title,
  genres_,
  actors_.actor,
  actors_.character
FROM 
  `bq-practice-477311.advanced.array_exercises`
CROSS JOIN  
  UNNEST(actors) AS actors_
CROSS JOIN
  UNNEST(genres) AS genres_
```

<img src='img/02-11.jpg' width=600>

<br>

#### 앱 로그 데이터

```sql
SELECT
  event_date,
  event_timestamp,
  event_name,
  event_param.key AS key,
  -- event_param.value AS value,
  event_param.value.string_value AS string_value,
  event_param.value.int_value AS int_value,
  -- event_params,
  user_id
FROM advanced.app_logs
CROSS JOIN UNNEST(event_params) AS event_param
```

<img src='img/02-12.jpg' width=600>

<br>



<br>

<hr>

<br>

## `PIVOT`
- `PIVOT` : 축을 중심으로 회전

<br>

### `PIVOT` 쿼리
- `MAX`, `IF`, `GROUP BY` 를 사용한 `PIVOT`
    - `SUM`, `COUNT` 등 다른 연산 사용 가능
- `PIVOT` 할 때, 모든 값이 같은 경우, `MAX`를 사용하거나 `ANY_VALUE`를 사용

<br>

<img src='img/02-13.png' width=600>

<br>

### `PIVOT` 예시

<br>

#### `user_id`별, `order_date`별로 주문이 있다면 1, 없다면 0으로 `PIVOT`
- `user_id`가 행, `order_date`가 열

<br>

```sql
SELECT
  user_id, 
  MAX(IF(order_date = '2023-05-01', 1, 0)) AS `2023-05-01`,
  MAX(IF(order_date = '2023-05-02', 1, 0)) AS `2023-05-02`,
  MAX(IF(order_date = '2023-05-03', 1, 0)) AS `2023-05-03`,
  MAX(IF(order_date = '2023-05-04', 1, 0)) AS `2023-05-04`,
  MAX(IF(order_date = '2023-05-05', 1, 0)) AS `2023-05-05`,
FROM advanced.orders
GROUP BY user_id;
```

<img src='img/02-14.png' width=600>

<br>

<br>

#### 앱 로그 데이터 배열 `PIVOT`
- 특정 `user_id`의 사용자가 `click_cart`를 누를 때, 어떤 `food_id`가 장바구니에 담겼는가?
- Key를 열로, Value들을 열 값으로 설정 하는 것이 필요

<br>

- **`GROUP BY ALL`은 `SELECT` 절에 등장하는 모든 비집계(non-aggregated) 컬럼을 자동으로 `GROUP BY`에 포함**

```sql
SELECT
  event_date,
  event_timestamp,
  event_name,
  user_id,
  user_pseudo_id,
  MAX(IF(ep.key = "firebase_screen", ep.value.string_value, NULL)) AS firebase_screen,
  MAX(IF(ep.key = "food_id", ep.value.int_value, NULL)) AS food_id,
  MAX(IF(ep.key = "session_id", ep.value.string_value, NULL)) AS session_id
FROM advanced.app_logs
CROSS JOIN UNNEST(event_params) AS ep
WHERE event_date BETWEEN  "2022-08-01" AND "2022-08-07"
GROUP BY ALL
```

<img src='img/02-15.png' width=800>

<br>

<hr>
